In [1]:
import cv2, numpy as np
import time
import theano
import keras
import os
from collections import OrderedDict, defaultdict
import six.moves.cPickle as pkl
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, LSTM, Lambda, merge
from keras.layers import Embedding
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D 
from keras.optimizers import SGD
from keras.layers import GRU, TimeDistributed, RepeatVector, Merge, TimeDistributedDense
import h5py
import json
from collections import Counter
import matplotlib.pyplot as plt
import keras.backend as K
from attention_lstm import AttentionLSTM
from keras.regularizers import l2
from keras.callbacks import Callback
#import theano.tensor as T
#import skimage.transform

Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 45.0% of memory, cuDNN 5105)
Using Theano backend.


In [2]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))
SEQUENCE_LENGTH = 45
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
BATCH_SIZE = 20
OUTPUT_DIM = 512
ANNOTATION_DIM = 512
WORD_DIM = 512
ANNOTATION_SIZE=196
ANNOTION_LENGTH = 196

In [3]:
d = []
with open('features/flickr30kann_1000_p3.pkl', 'rb') as f:
    d = pkl.load(f)
#pkl.dump(d, open('features/flickr30kann_1000_p3.pkl', 'wb'), protocol=pkl.HIGHEST_PROTOCOL)
vocab = d['vocab']
word_to_index = d['word_to_index']
index_to_word = d['index_to_word']

In [4]:
ins = d['INS']
gts = d['GTS']
ann_vec = d['AnnotationVectors']
ann_vec = ann_vec.reshape(1000,196,512)
z_mean = np.mean(ann_vec, axis=1)
print(ins.shape)
print(gts.shape)
print(ann_vec.shape)
print(z_mean.shape)
VOCAB_COUNT = len(vocab)

(1000, 44, 2452)
(1000, 44, 2452)
(1000, 196, 512)
(1000, 512)


In [5]:
ins_train = ins[0:900, :, :]
gts_train = gts[0:900, :, :]
ann_vec_train = ann_vec[0:900, :, :]
z_mean_train = z_mean[0:900, :]

ins_test = ins[900:, :, :]
gts_test = gts[900:, :, :]
ann_vec_test = ann_vec[900:, :, :]
z_mean_test = z_mean[900:, :]

x_train = [ins_train, ann_vec_train, z_mean_train]
x_test = [ins_test, ann_vec_test, z_mean_test]

y_train = gts_train
y_test = gts_test
print(ins_train.shape)
print(ins_test.shape)
print(z_mean_train.shape)
print(gts_test.shape)
print(ann_vec_train.shape)

(900, 44, 2452)
(100, 44, 2452)
(900, 512)
(100, 44, 2452)
(900, 196, 512)


In [6]:
aLstm_Layer = None

print('Building Model')
x_inp = Input(shape=(SEQUENCE_LENGTH-1, VOCAB_COUNT))
z_inp = Input(shape=(ANNOTATION_SIZE, ANNOTATION_DIM,))
z_mean = Input(shape=(ANNOTATION_DIM,))
h_Dense_l = Dense(OUTPUT_DIM, input_dim=ANNOTATION_DIM, activation='softmax')
h_Dense = h_Dense_l(z_mean)
c_Dense = Dense(OUTPUT_DIM, input_dim=ANNOTATION_DIM, activation='softmax')(z_mean)
#emb = Embedding(VOCAB_COUNT, WORD_DIM, input_length=SEQUENCE_LENGTH-1)(x_inp)
#tdemb = TimeDistributed()(emb)
xt_dense = TimeDistributed(Dense(WORD_DIM))(x_inp)
print(h_Dense)
print(c_Dense)
#print(tdemb)
#merge1 = merge([emb, h_Dense, c_Dense, z_inp], mode='concat', concat_axis=-1)
#global aLstm_Layer
aLstm_Layer = AttentionLSTM( output_dim=WORD_DIM, z_dim=ANNOTATION_DIM, W_regularizer=l2(0.01), U_regularizer=l2(0.01), Z_regularizer=l2(0.01), dropout_W=0.3, dropout_U=0.3, dropout_Z=0.3, return_sequences=True)
aLstm = aLstm_Layer([xt_dense, h_Dense, c_Dense, z_inp])
#aLstm_Alpha_Layer = AttentionLSTM(ret_alpha=True, output_dim=ANNOTION_LENGTH, z_dim=ANNOTATION_DIM, W_regularizer=l2(0.01), U_regularizer=l2(0.01), Z_regularizer=l2(0.01), dropout_W=0.3, dropout_U=0.3, dropout_Z=0.3, return_sequences=True)
#aLstm_Alpha = aLstm_Alpha_Layer([xt_dense, h_Dense, c_Dense, z_inp])
tdense = TimeDistributed(Dense(VOCAB_COUNT))(aLstm)
act = Activation('softmax')(tdense)
model = Model(input=[x_inp, z_inp, z_mean], output=act)
'''
model = Sequential()
print('Adding Embedding')
model.add(Embedding(VOCAB_COUNT, EMBEDDING_SIZE, input_length=SEQUENCE_LENGTH-1))
print('Adding LSTM')
model.add(AttentionLSTM(EMBEDDING_SIZE, Z_DIM, return_sequences=True))
print('Adding TimeDistributed Dense')
model.add(TimeDistributed(Dense(EMBEDDING_SIZE)))
return model
model.add(Dense(VOCAB_COUNT, activation='softmax'))
'''
#print(model.summary())
#    return model

Building Model
Softmax.0
Softmax.0
input shapes [(None, 44, 512), (None, 512), (None, 512), (None, 196, 512)]


"\nmodel = Sequential()\nprint('Adding Embedding')\nmodel.add(Embedding(VOCAB_COUNT, EMBEDDING_SIZE, input_length=SEQUENCE_LENGTH-1))\nprint('Adding LSTM')\nmodel.add(AttentionLSTM(EMBEDDING_SIZE, Z_DIM, return_sequences=True))\nprint('Adding TimeDistributed Dense')\nmodel.add(TimeDistributed(Dense(EMBEDDING_SIZE)))\nreturn model\nmodel.add(Dense(VOCAB_COUNT, activation='softmax'))\n"

In [7]:
class AlphaCallback(Callback):
    def on_train_begin(self, logs={}):
        self.alphaz = []

    def on_batch_end(self, batch, logs={}):
        self.alphaz.append(K.get_value(aLstm_Layer.states[3]))

In [7]:
#model = build_model()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 44, 2452)      0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 512)           0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           262656      input_3[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           262656      input_3[0][0]                    
___________________________________________________________________________________________

In [8]:
#print(keras.__version__)
#alphac = AlphaCallback()
his = model.fit(x_train, y_train, batch_size=3, nb_epoch=200, verbose=1)

Epoch 1/200
900/900 [==============================] - 55s - loss: 6.1549 - acc: 0.6109    
Epoch 2/200
900/900 [==============================] - 55s - loss: 2.1918 - acc: 0.6626    
Epoch 3/200
900/900 [==============================] - 53s - loss: 2.0607 - acc: 0.6728    
Epoch 4/200
900/900 [==============================] - 52s - loss: 1.9872 - acc: 0.6783    
Epoch 5/200
900/900 [==============================] - 53s - loss: 1.9243 - acc: 0.6838    
Epoch 6/200
900/900 [==============================] - 53s - loss: 1.8787 - acc: 0.6871    
Epoch 7/200
900/900 [==============================] - 54s - loss: 1.8410 - acc: 0.6906    
Epoch 8/200
900/900 [==============================] - 54s - loss: 1.8053 - acc: 0.6947    
Epoch 9/200
900/900 [==============================] - 55s - loss: 1.7820 - acc: 0.6962    
Epoch 10/200
900/900 [==============================] - 54s - loss: 1.7478 - acc: 0.6997    
Epoch 11/200
900/900 [==============================] - 54s - loss: 1.7081 - ac

In [14]:
print(model.layers[6].__dict__)

{'z_dim': 512, 'consume_less': 'cpu', 'ret_alpha': False, '_non_trainable_weights': [], 'dropout_Z': 0.3, 'd_regularizer': None, 'U_o': attentionlstm_1_U_o, 'regularizers': [<keras.regularizers.WeightRegularizer object at 0x000000001BE6A4E0>], 'W_regularizer': None, 'b2_att': attentionlstm_1_b2_att, 'dropout_U': 0.0, 'c_regularizer': None, 'forget_bias_init': <function one at 0x0000000009590EA0>, 'stateful': False, 'U': Join.0, 'Z_f': attentionlstm_1_Z_f, 'Wc_att': attentionlstm_1_Wc_att, 'c_f': attentionlstm_1_c_f, 'd_f': attentionlstm_1_d_f, 'uses_learning_phase': True, 'inner_init': <function orthogonal at 0x0000000009590D08>, 'Z': Join.0, 'b_i': attentionlstm_1_b_i, 'init': <function glorot_uniform at 0x0000000009590B70>, 'b_regularizer': None, 'W': Join.0, 'd_c': attentionlstm_1_d_c, 'U_i': attentionlstm_1_U_i, 'c_o': attentionlstm_1_c_o, 'Z_i': attentionlstm_1_Z_i, 'constraints': {}, 'd': Join.0, 'name': 'attentionlstm_1', 'outbound_nodes': [<keras.engine.topology.Node object at 

In [36]:
aLstm_AlphaLayer = AttentionLSTM( ret_alpha=True, output_dim=WORD_DIM, z_dim=ANNOTATION_DIM, W_regularizer=l2(0.01), U_regularizer=l2(0.01), Z_regularizer=l2(0.01), dropout_W=0.3, dropout_U=0.3, dropout_Z=0.3, return_sequences=True, weights=model.layers[6].get_weights())
aLstm_Alpha = aLstm_AlphaLayer([xt_dense, h_Dense, c_Dense, z_inp])
model2 = Model(input=[x_inp, z_inp, z_mean], output=aLstm_Alpha)

input shapes [(None, 44, 512), (None, 512), (None, 512), (None, 196, 512)]


In [37]:
y_pre = model2.predict(x_train)

In [40]:
print(y_pre.shape)
with open('results/alphab.pkl', 'wb') as f:
    pkl.dump(y_pre, f, protocol=pkl.HIGHEST_PROTOCOL)

(900, 44, 196)


In [15]:
prev_h1 = K.dot(z_mean, h_Dense_l.W) + h_Dense_l.b
proj_z = K.dot(z_inp, aLstm_Layer.Wc_att) + aLstm_Layer.b_att
proj_state = K.dot(prev_h1, aLstm_Layer.Wd_att)
proj_z = proj_z + proj_state[:, None, :]
proj_z = K.tanh(proj_z)

alpha = K.dot(proj_z, aLstm_Layer.U_att ) + aLstm_Layer.b2_att
alpha_shape = alpha.shape
alpha = K.softmax(alpha.reshape((alpha_shape[0], alpha_shape[1])))

f_alpha = K.function([z_inp, z_mean], alpha)

In [21]:
#alpha = f_alpha([ann_vec_train, z_mean_train])
#print(type(ins))
#al = alpha_func(x_train)
#print(al.shape)
#print(al)
with open('results/alpha.pkl', 'wb') as f:
    pkl.dump(alpha, f, protocol=pkl.HIGHEST_PROTOCOL)

In [10]:
model.fit(x_train, y_train, batch_size=3, nb_epoch=20, verbose=1)

Epoch 1/20


ValueError: GpuElemwise. Input dimension mis-match. Input 1 (indices start at 0) has shape[0] == 1, but the output's size on that axis is 3.
Apply node that caused the error: GpuElemwise{Add}[(0, 0)](GpuCAReduce{add}{0,0,1}.0, <CudaNdarrayType(float32, matrix)>)
Toposort index: 118
Inputs types: [CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(3, 196), (1, 196)]
Inputs strides: [(196, 1), (0, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuDimShuffle{0,x,1,x}(GpuElemwise{Add}[(0, 0)].0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.
Apply node that caused the error: forall_inplace,gpu,grad_o

In [9]:
loss = model.evaluate(x_test, y_test, batch_size=1, verbose=1)

100/100 [==============================] - 4s     


In [10]:
print(loss)

[3.9598917388916015, 0.63886363700032234]


In [16]:
#y_pred = model.predict(x_train, batch_size=3, verbose=1)
y_pred_test = model.predict(x_train, batch_size=1, verbose=1)

900/900 [==============================] - 37s    


In [12]:
#print(y_pred.shape)

def gen_captions(y_p):
    sentences = []
    for y_i in y_p:
        sentence = ""
        for word_v in y_i:
            max_i = np.argmax(word_v, axis = -1)
            word = index_to_word[max_i]
            if word == '#END#' or word == '#NULL#':
                break
            sentence = sentence + word + " "
        #print(sentence)
        sentences.append(sentence)
    return sentences

In [29]:
gts_sentences = gen_captions(gts)
gts_train = gts_sentences[0:900]
gts_test = gts_sentences[900:]

def create_json_dict(s_train):
    s_d = []
    for i, sent in enumerate(s_train):
        dict_cap = {}
        dict_cap["image_id"] = i
        dict_cap["caption"] = sent
        s_d.append(dict_cap)
    return s_d

train_d = create_json_dict(gts_train)
test_d = create_json_dict(gts_test)
print(gts_train[0:50])

['Two young guys with shaggy hair look at their hands while hanging out in the yard ', 'Several men in hard hats are operating a giant pulley system ', 'A child in a pink dress is climbing up a set of stairs in an entry way ', 'Someone in a blue shirt and hat is standing on stair and leaning against a window ', 'Two men one in a gray shirt one in a black shirt standing near a stove ', 'Two people in the photo are playing the guitar and the other is poking at him ', 'A man sits in a chair while holding a large stuffed animal of a lion ', 'A girl is on rollerskates talking on her cellphone standing in a parking lot ', 'An asian man wearing a black suit stands near a darkhaired woman and a brownhaired woman ', 'Two men in Germany jumping over a rail at the same time without shirts ', 'Five ballet dancers caught mid jump in a dancing studio with sunlight coming through a window ', 'Three young men and a young woman wearing sneakers are leaping in midair at the top of a flight of concrete s

In [35]:
string = "The man wearing white clothes and white leg padding is practicing a sport"
for i, gts_str in enumerate(gts_sentences):
    if "leap" in gts_str:
        print("%s %s", (i, gts_str))

%s %s (11, 'Three young men and a young woman wearing sneakers are leaping in midair at the top of a flight of concrete stairs ')
%s %s (605, 'A dog leaps into the air in a grassy field surrounded by trees ')
%s %s (791, 'A dog takes a leap of faith as he jumps into a pool to grab an orange toy ')
%s %s (860, 'A goggled boy leaps from the edge into a shimmering pool with someone watching on in the background ')


In [22]:
import json

#train_full = create_json_dict(gts_sentences)
pred_sent_full = gen_captions(y_pred_test)
#pred_full = create_json_dict(pred_sent_full)

with open('results/attn_pred.txt', 'w') as f:
    f.writelines(pred_sent_full)
'''    
with open('results/attn_ref.json', 'w') as f:
    json.dump(train_full, f)
'''
print(pred_sent_full[0])

Two young guys with shaggy hair look at their hands while hanging out in the yard 


In [67]:
pred_sentence = []
pred_sentence = gen_captions(y_pred)
pred_d = create_json_dict(pred_sentence)
print(len(pred_sentence))
print(len(pred_d))
print(len(y_pred))

pred_test_sentence = []
pred_test_sentence = gen_captions(y_pred_test)
pred_d_test = create_json_dict(pred_test_sentence)
print(len(pred_test_sentence))
print(len(pred_d_test))
print(len(y_pred_test))

899
899
899
100
100
100


In [ ]:
y_pred = model.predict(x_train, batch_size=3, verbose=1)

In [68]:

with open('results/ref_train.json', 'w') as f:
    json.dump(train_d, f)

with open('results/ref_test.json', 'w') as f:
    json.dump(test_d, f)

with open('results/pred_train.json', 'w') as f:
    json.dump(pred_d, f)

with open('results/pred_test.json', 'w') as f:
    json.dump(pred_d_test, f)

In [79]:
j = []
with open('results/VGG16_LTSM_1k.pkl', 'rb') as f: 
    j = pkl.load(f, encoding='latin1')

def gen_json_dict_from_pkl(pkl_list):
    pred_list = []
    ref_list = []
    for i, pkl_dict in enumerate(pkl_list):
        dict_cap = {}
        dict_cap ['image_id'] = i
        dict_cap['caption'] = pkl_dict['predictions'][0]
        pred_list.append(dict_cap)
        dict_ref = {}
        dict_ref['image_id'] = i
        dict_ref['caption'] = pkl_dict['labels'][0]
        ref_list.append(dict_ref)
    return pred_list, ref_list

gnet_pred, gnet_ref = gen_json_dict_from_pkl(j)
with open('results/VGG16_LTSM_1k_pred.json', 'w') as f:
    json.dump(gnet_pred, f)

with open('results/VGG16_LTSM_1k_ref.json', 'w') as f:
    json.dump(gnet_ref, f)


In [8]:
def word_processing(dataset):
    allwords = Counter()
    for item in dataset:
        for sentence in item['sentences']:
            allwords.update(sentence['tokens'])
            
    vocab = [k for k, v in allwords.items()]
    vocab.insert(0, '<NULL>')
    vocab.append('<START>')
    vocab.append('<END>')
    vocab.append('<UNK>')

    word_to_index = {w: i for i, w in enumerate(vocab)}
    index_to_word = {i: w for i, w in enumerate(vocab)}
    return vocab, word_to_index, index_to_word

def import_flickr8kdataset():
    dataset = json.load(open('captions/dataset_flickr8k.json'))['images']
    #reduced length to a 300 for testing
    val_set = list(filter(lambda x: x['split'] == 'val', dataset))
    train_set = list(filter(lambda x: x['split'] == 'train', dataset))
    test_set = list(filter(lambda x: x['split'] == 'test', dataset))
    return train_set[:800]+val_set[:200]


In [9]:
def floatX(arr):
    return np.asarray(arr, dtype=theano.config.floatX)

#Prep Image uses an skimage transform
def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 224, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (224, w*224/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*224/w, 224), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [10]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


In [21]:
def language_model():
    model = Sequential()
    print('Adding Embedding')
    model.add(Embedding(VOCAB_COUNT, EMBEDDING_SIZE, input_length=SEQUENCE_LENGTH-1))
    print('Adding LSTM')
    model.add(LSTM(EMBEDDING_SIZE, return_sequences=False))
    #print('Adding TimeDistributed Dense')
    #model.add(TimeDistributed(Dense(EMBEDDING_SIZE)))
    return model

In [26]:
dataset = import_flickr8kdataset()
# Currently testing it out
dataset = [i for i in dataset[:100]]
vocab,word_to_index, index_to_word = word_processing(dataset)
#print vocab

In [27]:
def chunks(l, n):
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

def process_images(dataset, coco=False, d_set="Flicker8k_Dataset"):
    ind_process = 1
    total = len(dataset)
    cnn_input = floatX(np.zeros((len(dataset), 3, 224, 224)))
    rawim_input = []
    sentences_tokens = []
    for i, image in enumerate(dataset):
        print "ind_process %s total %s" %(str(ind_process),str(total))
        ind_process+=1
        if coco:
            fn = './coco/{}/{}'.format(image['filepath'], image['filename'])
        else:
            fn = d_set+'/{}'.format(image['filename'])
        try:
            im = plt.imread(fn)
            rawim, cnn_input[i] = prep_image(im)
            sentences_tokens.append(image['sentences'][0]['tokens'])
            rawim_input.append(rawim)
        except IOError:
            continue
    return rawim_input, cnn_input, sentences_tokens


In [29]:
#rawim_array, cnnim_array, sentences_tokens = process_images(dataset, coco=False, d_set="Flicker8k_Dataset")
%matplotlib inline

In [28]:
def gen_image_partial_captions(images, captions, word_to_index, vocab_count, model):
    a_images = []
    a_captions = []
    next_words = []
    #vocab_size = len(vocab)
    for ind, image in enumerate(images):
        sentence = captions[ind]
        partial_caption_ar = np.zeros(SEQUENCE_LENGTH-1, dtype=np.int)
        
        words = ['<START>'] + sentence + ['<END>']
        assert len(words)<SEQUENCE_LENGTH
        for i in range(len(words) - 1):
            pc_copy = partial_caption_ar.copy()
            if words[i] in word_to_index:
                pc_copy[i] = word_to_index[words[i]]
            else:
                pc_copy[i] = word_to_index["<UNK>"]
            a_images.append(image)
            a_captions.append(pc_copy)
            #Generate next word output vector
            next_word = words[i + 1]
            if next_word in word_to_index:
                next_word_index = word_to_index[next_word]
            else:
                next_word_index = word_to_index["<UNK>"]
            next_word_ar = np.zeros(vocab_count, dtype=np.int)
            next_word_ar[next_word_index] = 1
            next_words.append(next_word_ar)
    v_i = np.array(a_images)
    v_c = np.array(a_captions)
    v_nw = np.array(next_words)
    return v_i, v_c, v_nw 

In [30]:
#vocab_count = len(word_to_index)
#print cnnim_array.shape
#v_i, v_c, v_nw = gen_image_partial_captions(cnnim_array, sentences_tokens, word_to_index, vocab_count)

In [31]:
VOCAB_COUNT = len(word_to_index)

In [32]:
def chunks(l, n):
    for i in xrange(0, len(l), n):
        yield l[i:i + n]

def process_cnn_features(dataset, model, coco=False, d_set="Flicker8k_Dataset"):
    ind_process = 1
    total = len(dataset)
    for chunk in chunks(dataset, 25):
        cnn_input = floatX(np.zeros((len(chunk), 3, 224, 224)))
        for i, image in enumerate(chunk):
            print "ind_process %s total %s" %(str(ind_process),str(total))
            ind_process+=1
            if coco:
                fn = './coco/{}/{}'.format(image['filepath'], image['filename'])
            else:
                fn = d_set+'/{}'.format(image['filename'])
            try:
                im = plt.imread(fn)
                _, cnn_input[i] = prep_image(im)
            except IOError:
                continue
        features = model.predict(cnn_input)
        print "Processing Features For Chunk"
        for i, image in enumerate(chunk):
            image['cnn features'] = features[i]
            
def process_captions_array(dataset):
    ind_process = 1
    total = len(dataset)
    for chunk in chunks(dataset, 25):
        for i in chunk:
            
            image["captions"]

In [33]:
model = VGG_16('weights/vgg16_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')


In [ ]:
def predict(model, images, index_to_word, word_to_index):
    for image in images:
        caption = np.zeros(SEQUENCE_LENGTH - 1).reshape(1, SEQUENCE_LENGTH - 1)
        print(caption.shape)
        caption[0,0] = word_to_index["<START>"]
        count=0
        sentence = []
        a = image.reshape(1,3,224,224)
        #a = np.array([image])
        while True:
            out = model.predict([a, caption])
            index = out.argmax(-1)
            index = index[0]
            word = index_to_word[index]
            sentence.append(word)
            count+= 1
            if count >= SEQUENCE_LENGTH - 1 or index == word_to_index["<END>"]: #max caption length reach of '<eos>' encountered
                break
            caption[0,count] = index
        sent_str = " ".join(sentence)
        print("The Oracle says : %s" %sent_str)


In [ ]:
cnnim_list = []
for i in cnnim_array:
    cnnim_list.append(i)

In [ ]:
predict(model, cnnim_list, index_to_word, word_to_index)

In [ ]:
caption = np.zeros(SEQUENCE_LENGTH).reshape(1, SEQUENCE_LENGTH)
caption[0] = word_to_index["#START#"]
print cnnim_array[0].shape
t = np.array(cnnim_array[0])
print t.shape
out = model.predict([v_i, v_c])
print out
